In [26]:
import sys
print(sys.executable)

/Users/damian/mlops-proyecto-fase-1-equipo-29/.venv/bin/python


## **Etapa 2.** Procesamiento y transformación de datos

**Objetivos:**

* Realizar transformaciones necesarias para preparar los datos para el análisis y modelado.

* Realizar análisis exploratorio de datos utilizando visualizaciones y estadísticas descriptivas.

-------
Con el análisis exploratorio que realizamos en el paso 1, descubrimos que tenemos variables categóricas, numéricas y binarias, y somos concientes de que nos enfrentamos a un problema de Outliers en varias de las variables. 

Con ese conocimiento, éste punto pretende realizar la generación del pipeline para la transformación de los datos, en preparación para el entrenamiento del modelo.

* **Datos categóricos:** Aplicación de One Hot Enconder
* **Datos numéricos:** Estandarización de datos
* **Datos binarios:** Transformación de las categorías a 1 y 0.

----

Paqueterías usadas en este archivo

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import os

from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, cross_validate, train_test_split
from sklearn.metrics import fbeta_score, make_scorer, precision_recall_curve, PrecisionRecallDisplay, RocCurveDisplay, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

Lectura de la data set limpio, obtenido en el código 01: 

In [28]:
path_clean = "../../data/processed/data_clean.csv"

try:
    # Leer el CSV usando la ruta relativa
    data_clean = pd.read_csv(path_clean)

    print("Lectura exitosa del archivo de datos crudos.")
    print(f"Filas leídas: {len(data_clean)}")
    print("\nPrimeras 5 filas:")
    print(data_clean.head())

except FileNotFoundError:
    print(f"ERROR: No se pudo encontrar el archivo en la ruta: {path_clean}")
    print("Asegúrate de que la ruta relativa sea correcta desde la ubicación de este notebook.")
except Exception as e:
    print(f"Ocurrió un error inesperado durante la lectura: {e}")

Lectura exitosa del archivo de datos crudos.
Filas leídas: 992

Primeras 5 filas:
   laufkont  laufzeit  moral  verw   hoehe  sparkont  beszeit  rate  famges  \
0       1.0      18.0    4.0   2.0  1049.0       1.0      2.0   4.0     2.0   
1       1.0       9.0    4.0   0.0  2799.0       1.0      3.0   2.0     3.0   
2       2.0      12.0    2.0   9.0   841.0       2.0      4.0   2.0     2.0   
3       1.0      12.0    4.0   0.0  2122.0       1.0      3.0   3.0     3.0   
4       1.0      12.0    4.0   0.0  2171.0       1.0      3.0   4.0     3.0   

   buerge  ...  verm  alter  weitkred  wohn  bishkred  beruf  pers  telef  \
0     1.0  ...   2.0   21.0       3.0   1.0       1.0    3.0   2.0    1.0   
1     1.0  ...   1.0   36.0       3.0   1.0       2.0    3.0   1.0    1.0   
2     1.0  ...   1.0   23.0       3.0   1.0       1.0    2.0   2.0    1.0   
3     1.0  ...   1.0   39.0       3.0   1.0       2.0    2.0   1.0    1.0   
4     NaN  ...   2.0   38.0       1.0   2.0       2.0    2

In [29]:
# --- 1. Definición de Nombres de Columnas ---

# Variables Numéricas
numericas_pipe_nombres = ['laufzeit', 'hoehe', 'alter']

# Variables Categóricas-Nominales (que requieren One-Hot Encoding)
nominales_pipe_nombres = [
    'laufkont', # (status - categorical)
    'moral',    # (credit_history - categorical)
    'verw',     # (purpose - categorical)
    'sparkont', # (savings - categorical)
    'famges',   # (personal_status_sex - categorical)
    'buerge',   # (other_debtors - categorical)
    'weitkred', # (other_installment_plans - categorical)
    'wohn',     # (housing - categorical)
    'pers',     # (people_liable - binary)
    'telef',    # (telephone - binary)
    'gastarb'   # (foreign_worker - binary)
]

# Variables Categóricas-Ordinales (que requieren Ordinal Encoding)
ordinales_pipe_nombres = [
    'beszeit',  # (employment_duration - ordinal)
    'rate',     # (installment_rate - ordinal)
    'wohnzeit', # (present_residence - ordinal)
    'verm',     # (property - ordinal)
    'bishkred', # (number_credits - ordinal)
    'beruf'     # (job - ordinal)
]

In [30]:
## PIPELINE DE TRANSFORMACIÓN DE LOS DATOS (ÉSTA VERSIÓN SE PUEDE APLICAR SI DECIDIMOS ELIMINAR NULOS)

# Variables numéricas:
numericas_pipe = Pipeline(steps = [('impMediana', SimpleImputer(strategy='median')),
                                 ('escalaNum', MinMaxScaler(feature_range=(1,2)))])

# Variables categóricas-Nominales:
nominales_pipe = Pipeline(steps = [('impModa', SimpleImputer(strategy='most_frequent')),
                             ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore'))])

# Variables categóricas-ordinales:
ordinales_pipe = Pipeline(steps = [('impOrd', SimpleImputer(strategy='most_frequent')),
                                ('ordtrasnf', OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1))])

# Conjuntas las transformaciones de todo tipo de variable y
# deja sin procesar aquellas que hayas decidido no transformar:

columnasTransformer = ColumnTransformer(transformers = [('numpipe', numericas_pipe, numericas_pipe_nombres),
                                                        ('nominals', nominales_pipe, nominales_pipe_nombres),
                                                        ('ordinales', ordinales_pipe, ordinales_pipe_nombres)],
                                        remainder='passthrough')

In [31]:
## PIPELINE DE TRANSFORMACIÓN DE LOS DATOS (ÉSTA VERSIÓN SE PUEDE APLICAR SI DECIDIMOS IMPUTAR NULOS)

# --- 2. Definición de Pipelines Individuales (con Imputación) ---

# Pipeline para Variables Numéricas: Imputación con Mediana + Escalado MinMax
numericas_pipe = Pipeline(steps = [
    ('impMediana', SimpleImputer(strategy='median')),
    ('escalaNum', MinMaxScaler(feature_range=(1,2)))
])

# Pipeline para Variables Nominales: Imputación con Moda + One-Hot Encoding
nominales_pipe = Pipeline(steps = [
    ('impModa', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

# Pipeline para Variables Ordinales: Imputación con Moda + Ordinal Encoding
ordinales_pipe = Pipeline(steps = [
    ('impOrd', SimpleImputer(strategy='most_frequent')),
    ('ordtrasnf', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])


# --- 3. Combinación de Pipelines con ColumnTransformer ---

data_preprocessor = ColumnTransformer(
    transformers=[
        # Aplica la pipeline numérica a las columnas numéricas
        ('num', numericas_pipe, numericas_pipe_nombres),
        # Aplica la pipeline nominal a las columnas nominales
        ('nom', nominales_pipe, nominales_pipe_nombres),
        # Aplica la pipeline ordinal a las columnas ordinales
        ('ord', ordinales_pipe, ordinales_pipe_nombres)
    ],
    remainder='passthrough', # Mantener columnas que no estén en la lista (si las hay)
    verbose_feature_names_out=False # Simplifica los nombres de las features de salida
).set_output(transform="pandas")


# --- 4. Función de Preprocesamiento Principal ---

def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aplica el ColumnTransformer completo al DataFrame, realizando la imputación,
    escalado y codificación necesarios para la implementación del modelo.

    Args:
        df (pd.DataFrame): El DataFrame con las variables a preprocesar.

    Returns:
        pd.DataFrame: El DataFrame transformado (o None si hay un error).
    """
    print("\n--- Iniciando Preprocesamiento de Datos ---")

    try:
        # Ajusta y transforma los datos
        # Nota: Si ya tienes un transformador ajustado (fit), usa solo .transform(df)
        df_processed = data_preprocessor.fit_transform(df)

        print("Preprocesamiento completado. Datos listos para el modelo.")
        print(f"Shape de los datos transformados: {df_processed.shape}")

        return df_processed

    except Exception as e:
        print(f"Error durante el preprocesamiento: {e}")
        return None

**División de la base entre entrenamiento y validación**

* Variable objetivo (y): 'kredit'
* El análisis muestra que hay un desbalanceo significativo entre las clases, pues la proporción de datos es 70:30, con lo cual podemos considerar que hay una diferencia considerable entre las clases de buenos y malos en el conjunto de datos.

Éste desequilibrio puede afectar al rendimiento de los modelos de clasificación que podamos plantear, especialmente para el caso de la clase minoritaria. Para tratar de lidear con el problema, a lo largo del ejercicio se plantearán alternativas como el sobreajuste y subajuste, que nos apoyará a equilibrar el conjunto de datos, dándo mayor representación a la clase minoritatía y apoyando al desempeño del modelo.

In [32]:
data_clean['kredit'].value_counts(normalize = True)

kredit
1.0    0.694556
0.0    0.305444
Name: proportion, dtype: float64

In [33]:
def split_data_by_target(df: pd.DataFrame, target_column_name: str):
    """
    Divide un DataFrame en dos partes:
    X (variables predictoras/features) y y (variable objetivo/target).

    Args:
        df (pd.DataFrame): El DataFrame completo que contiene features y target.
        target_column_name (str): El nombre de la columna que será la variable 'y'.

    Returns:
        tuple: Una tupla que contiene (X, y), donde X es un DataFrame y y es una Serie.
               Devuelve (None, None) si la columna target no existe.
    """
    # Verificar si la columna target existe en el DataFrame
    if target_column_name not in df.columns:
        print(f"Error: La columna objetivo '{target_column_name}' no se encontró en el DataFrame.")
        return None, None

    # Definir la variable objetivo (y)
    y = df[target_column_name].copy()

    # Definir las variables predictoras (X)
    X = df.drop(columns=[target_column_name], axis=1).copy()

    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, stratify= y, random_state=1234) # La semilla garantiza que nuestro código sea reproducible

    print(f"División de datos completada.")
    print(f"   - X Train (Features): {Xtrain.shape[0]} filas, {Xtrain.shape[1]} columnas.")
    print(f"   - X Test (Features): {Xtest.shape[0]} filas, {Xtest.shape[1]} columnas.")
    print(f"   - y Train (Target '{target_column_name}'): {ytrain.shape[0]} filas.")
    print(f"   - y Test (Target '{target_column_name}'): {ytest.shape[0]} filas.")

    return Xtrain, Xtest, ytrain, ytest


In [34]:
## Ejecución del código para dividir entre train y test

Xtrain, Xtest, ytrain, ytest = split_data_by_target(data_clean, 'kredit')

División de datos completada.
   - X Train (Features): 694 filas, 20 columnas.
   - X Test (Features): 298 filas, 20 columnas.
   - y Train (Target 'kredit'): 694 filas.
   - y Test (Target 'kredit'): 298 filas.


In [35]:
## Ejecutamos la transformación con el conjunto de entrenamiento

Xtmp = Xtrain.copy()
tmp = columnasTransformer.fit_transform(Xtmp)
print("Dimensión de los datos de entrada:")
print("antes de aplicar las transformaciones:", Xtmp.shape)

print("después de aplicar las transformaciones:", tmp.shape)

Dimensión de los datos de entrada:
antes de aplicar las transformaciones: (694, 20)
después de aplicar las transformaciones: (694, 123)


In [36]:
# Una vez ejecutada la transformación, y como se va a utilizar Validación-Cruzada, concatena los conjuntos de entrenamiento
# y prueba en uno nuevo conjunto aumentado que llamaremos trainval:

Xtraintest = pd.concat([Xtrain, Xtest], axis=0)
ytraintest = pd.concat([ytrain, ytest], axis=0)

# Veamos cuántas variables nuevas se introducen con las transformaciones One-Hot-Encoding:
Xtmp = Xtraintest.copy()
tmp = columnasTransformer.fit_transform(Xtmp)
print("Dimensión de las variables de entrada ANTES de las transformaciones:", Xtmp.shape)
print("Dimensión de las variables de entrada DESPUÉS de las transformaciones:", tmp.shape)

Dimensión de las variables de entrada ANTES de las transformaciones: (992, 20)
Dimensión de las variables de entrada DESPUÉS de las transformaciones: (992, 154)


### Almacenamiento de salida final:

In [37]:
Xtraintest.to_csv(os.path.join('..', '..', 'data', 'processed', 'Xtraintest.csv'), index=False)
ytraintest.to_csv(os.path.join('..', '..', 'data', 'processed', 'ytraintest.csv'), index=False)